# **Introduction**

Wandb workspace URL: https://wandb.ai/fabian-dubach-hochschule-luzern/experiment-tracking/runs/gtg68kwg/workspace?nw=nwuserfabiandubach

# **Setup**


### WandDB

In [1]:
import wandb
from lightning.pytorch.loggers import WandbLogger

wandb.login(key="")
wandb_logger = WandbLogger(project="experiment-tracking")

c:\Users\fabia\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.


In [2]:

def init_wandb_run(project_name, run_name, config_dict):
    run = wandb.init(
        project=project_name,
        name=run_name,
        config=config_dict,
    )
    return run


#### Example Run
experiment_1_config = {
    "learning_rate": 0.001,
    "epochs": 10,
    "batch_size": 32,
    "optimizer": "adam"
}
run1 = init_wandb_run("experiment-tracking", "experiment_1", experiment_1_config)


wandb: Currently logged in as: fabian-dubach (fabian-dubach-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


# **Preprocessing**

### Import all libraries needed

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from datasets import load_dataset
import numpy as np

### Load the dataset from HuggingFace

In [4]:
dataset = load_dataset("tau/commonsense_qa")

### Load Word2Vec

In [5]:
import gensim.downloader as api
wv = api.load("glove-wiki-gigaword-100")

### Load BERT tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [7]:
def get_embedding(text):
    tokens = tokenizer.tokenize(text)
    vectors = [wv[word] if word in wv else np.zeros(100) for word in tokens]
    return np.mean(vectors, axis=0)

In [8]:
class CommonsenseQADataset(Dataset):
    def __init__(self, dataset, split="train"):
        self.data = dataset[split]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data[idx]["question"]
        choices = self.data[idx]["choices"]["text"]
        label = self.data[idx]["answerKey"]
        
        embeddings = [get_embedding(question + " " + choice) for choice in choices]
        embeddings = torch.tensor(embeddings, dtype=torch.float)
        
        label_idx = ord(label) - ord('A')
        return embeddings, label_idx

In [9]:
train_dataset = CommonsenseQADataset(dataset, "train")
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [10]:
class Classifier(nn.Module):
    def __init__(self, embedding_dim=100, hidden_dim=128, num_classes=5):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
for epoch in range(10):
    for embeddings, labels in train_loader:
        embeddings, labels = embeddings.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

C:\Users\fabia\AppData\Local\Temp\ipykernel_3496\2918860721.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  embeddings = torch.tensor(embeddings, dtype=torch.float)


RuntimeError: Expected target size [32, 5], got [32]

# Model

# Training

# Evaluation

# Interpretation